In [1]:
#!/usr/bin/env python
# coding: utf-8

if 'config' not in locals():
    config = 'configs.config_ILLAE_local'
#     config = 'configs.config_WomenInEngineering_local'
#     config = 'configs.config_Digits_local'    

In [2]:
### initialization ###

import importlib
print('config file is set to {}'.format(config))

imp=importlib.import_module('_imports')
importlib.reload(imp)

util=importlib.import_module('_utils')
importlib.reload(util)

c=importlib.import_module(config)
importlib.reload(c)

get_ipython().run_line_magic('matplotlib', 'inline')

# param_setup(sys.argv[1:], c)
util.param_setup_ipython(globals(), c)

config file is set to configs.config_ILLAE_local


In [3]:
### reading the dataset file ###

dfs=dict()
dfs['input']=imp.pd.read_csv('{}data/{}_1_data_filtered.csv'.format(c.directory['save'], c.project_name))

imp.pprint(dfs['input'].shape)
util.display_df(dfs['input'].head())

(17354, 34)


,in_reply_to_status_id,in_reply_to_screen_name,in_reply_to_user_id,retweet_count,favorite_count,tweet_created_at_date,tweet_created_at_hour,tweet_id,text,source,coordinates,name,screen_name,verified,friends_count,followers_count,favourites_count,statuses_count,listed_count,description,profile_created_at_date,profile_created_at_hour,location,language,user_id,profile_image_url_https,profile_background_image_url_https,profile_banner_url,external_url,user_mentions,hashtags,external_media_type,external_media_url,date_converted
0,NaN,NaN,NaN,0,0,Aug 27 2015,20:55:52,637005633548632064,#iLookLikeAnEngineer: Nuance employees share s...,"<a href=""http://socialchorus.com"" rel=""nofollo...",NaN,Nuance UK,NuanceUK,0,34,946,78,1127,51,"Nuance UK - the home of Dragon, PowerPDF and O...",May 06 2009,10:13:26,UK,en,38150806,https://pbs.twimg.com/profile_images/677196289...,https://abs.twimg.com/images/themes/theme9/bg.gif,https://pbs.twimg.com/profile_banners/38150806...,http://bit.ly/1PW3R9T,NaN,"iLookLikeAnEngineer, nuanceteam",NaN,NaN,2015-08-27
1,NaN,NaN,NaN,1,0,Aug 27 2015,21:02:37,637007331574812673,We're so proud of our mentors. Earlier this mo...,"<a href=""http://www.hootsuite.com"" rel=""nofoll...",NaN,SparkProgramLA,SparkProgramLA,0,196,253,117,672,11,"Every day, we foster relationships between mid...",Sep 11 2013,17:42:45,"Los Angeles, CA",en,1854956568,https://pbs.twimg.com/profile_images/719566009...,https://pbs.twimg.com/profile_background_image...,https://pbs.twimg.com/profile_banners/18549565...,NaN,"1719657061, 135266722",ILookLikeAnEngineer,NaN,NaN,2015-08-27
2,NaN,NaN,NaN,1,3,Aug 27 2015,21:24:57,637012950495031296,#Ilooklikeanengineer http://t.co/Zrz7y3Vt3X,"<a href=""http://www.apple.com"" rel=""nofollow"">...",NaN,kate,Szkolnik,0,1287,13,2,3,2,NaN,May 06 2011,13:22:01,NaN,en,294060143,https://pbs.twimg.com/profile_images/134470334...,https://abs.twimg.com/images/themes/theme1/bg.png,NaN,NaN,NaN,Ilooklikeanengineer,photo,https://pbs.twimg.com/media/CNcf5zwUEAAXhBC.jpg,2015-08-27
3,NaN,kstate_wie,2.677363e+09,2,4,Aug 27 2015,21:25:08,637012998473670656,@kstate_wie @KStateEngg #ILookLikeAnEngineer h...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,Spency Spenc,spencer_slane,0,464,494,2341,8345,2,Mechanical Engineer - BAJA TREASURER,Oct 02 2011,14:36:49,KSU,en,383790528,https://pbs.twimg.com/profile_images/780423411...,https://abs.twimg.com/images/themes/theme1/bg.png,https://pbs.twimg.com/profile_banners/38379052...,NaN,"2677362781, 32296630",ILookLikeAnEngineer,photo,https://pbs.twimg.com/media/CNcf7_XVEAAjLQg.jpg,2015-08-27
4,NaN,NaN,NaN,0,0,Aug 27 2015,21:32:03,637014738216443905,#ILookLikeAnEngineer former yoga teacher turne...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,Manu Lohiya,manu_lohiya,0,416,206,16,85,7,A user-driven Product Manager that can code an...,Dec 14 2012,00:38:01,"San Francisco, CA",en,1010066696,https://pbs.twimg.com/profile_images/333657966...,https://pbs.twimg.com/profile_background_image...,NaN,NaN,170393291,"ILookLikeAnEngineer, thisissf",photo,https://pbs.twimg.com/media/CNchht-UAAAhCgO.jpg,2015-08-27


In [4]:
### creating label dataframe + saving ###

# dfs['label'] = imp.pd.DataFrame()
# dfs['input'][c.id_name] = dfs['input'][c.id_name]
if c.label is not None:
    if c.label['type'] == 'boolean':
        dfs['input']['label'] = dfs['input'][c.label['col']] > 0
        dfs['input']['label'] = dfs['input']['label'].astype(str)

    dfs['input'][[c.id_name, 'label']].to_csv('{}data/{}_2_data_labels.csv'.format(c.directory['save'], c.project_name), index=False)    
    util.display_df(dfs['input'].head(5))

,in_reply_to_status_id,in_reply_to_screen_name,in_reply_to_user_id,retweet_count,favorite_count,tweet_created_at_date,tweet_created_at_hour,tweet_id,text,source,coordinates,name,screen_name,verified,friends_count,followers_count,favourites_count,statuses_count,listed_count,description,profile_created_at_date,profile_created_at_hour,location,language,user_id,profile_image_url_https,profile_background_image_url_https,profile_banner_url,external_url,user_mentions,hashtags,external_media_type,external_media_url,date_converted,label
0,NaN,NaN,NaN,0,0,Aug 27 2015,20:55:52,637005633548632064,#iLookLikeAnEngineer: Nuance employees share s...,"<a href=""http://socialchorus.com"" rel=""nofollo...",NaN,Nuance UK,NuanceUK,0,34,946,78,1127,51,"Nuance UK - the home of Dragon, PowerPDF and O...",May 06 2009,10:13:26,UK,en,38150806,https://pbs.twimg.com/profile_images/677196289...,https://abs.twimg.com/images/themes/theme9/bg.gif,https://pbs.twimg.com/profile_banners/38150806...,http://bit.ly/1PW3R9T,NaN,"iLookLikeAnEngineer, nuanceteam",NaN,NaN,2015-08-27,False
1,NaN,NaN,NaN,1,0,Aug 27 2015,21:02:37,637007331574812673,We're so proud of our mentors. Earlier this mo...,"<a href=""http://www.hootsuite.com"" rel=""nofoll...",NaN,SparkProgramLA,SparkProgramLA,0,196,253,117,672,11,"Every day, we foster relationships between mid...",Sep 11 2013,17:42:45,"Los Angeles, CA",en,1854956568,https://pbs.twimg.com/profile_images/719566009...,https://pbs.twimg.com/profile_background_image...,https://pbs.twimg.com/profile_banners/18549565...,NaN,"1719657061, 135266722",ILookLikeAnEngineer,NaN,NaN,2015-08-27,True
2,NaN,NaN,NaN,1,3,Aug 27 2015,21:24:57,637012950495031296,#Ilooklikeanengineer http://t.co/Zrz7y3Vt3X,"<a href=""http://www.apple.com"" rel=""nofollow"">...",NaN,kate,Szkolnik,0,1287,13,2,3,2,NaN,May 06 2011,13:22:01,NaN,en,294060143,https://pbs.twimg.com/profile_images/134470334...,https://abs.twimg.com/images/themes/theme1/bg.png,NaN,NaN,NaN,Ilooklikeanengineer,photo,https://pbs.twimg.com/media/CNcf5zwUEAAXhBC.jpg,2015-08-27,True
3,NaN,kstate_wie,2.677363e+09,2,4,Aug 27 2015,21:25:08,637012998473670656,@kstate_wie @KStateEngg #ILookLikeAnEngineer h...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,Spency Spenc,spencer_slane,0,464,494,2341,8345,2,Mechanical Engineer - BAJA TREASURER,Oct 02 2011,14:36:49,KSU,en,383790528,https://pbs.twimg.com/profile_images/780423411...,https://abs.twimg.com/images/themes/theme1/bg.png,https://pbs.twimg.com/profile_banners/38379052...,NaN,"2677362781, 32296630",ILookLikeAnEngineer,photo,https://pbs.twimg.com/media/CNcf7_XVEAAjLQg.jpg,2015-08-27,True
4,NaN,NaN,NaN,0,0,Aug 27 2015,21:32:03,637014738216443905,#ILookLikeAnEngineer former yoga teacher turne...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,Manu Lohiya,manu_lohiya,0,416,206,16,85,7,A user-driven Product Manager that can code an...,Dec 14 2012,00:38:01,"San Francisco, CA",en,1010066696,https://pbs.twimg.com/profile_images/333657966...,https://pbs.twimg.com/profile_background_image...,NaN,NaN,170393291,"ILookLikeAnEngineer, thisissf",photo,https://pbs.twimg.com/media/CNchht-UAAAhCgO.jpg,2015-08-27,False


In [5]:
### calculating stratified outer + inner folds and saving ###

dfs['input']=imp.pd.read_csv('{}data/{}_2_data_labels.csv'.format(c.directory['save'], c.project_name))
class_labels = set(dfs['input']['label'])

samples=min(dfs['input']['label'].value_counts())
print('Number of samples per each class = {}'.format(samples))
util.mkdir(c.directory['save']+'cv_ids/')

ids_list=list()
labels_list=list()

for label in class_labels:
    ids=list(imp.np.random.choice(dfs['input'].loc[dfs['input']['label']==label, c.id_name], samples, replace=False))
    labels=[label]*samples
    ids_list.extend(ids)
    labels_list.extend(labels)
    
skf_outer = imp.StratifiedKFold(n_splits=c.outer, random_state=c.seed)
for outer_index, (train_index, test_index) in enumerate(skf_outer.split(ids_list, labels_list)):
#     print (test_index)

    outer_ids = [ids_list[id] for id in test_index]
    outer_labels = [labels_list[id] for id in test_index]
    inner_ids = [ids_list[id] for id in train_index]
    inner_labels = [labels_list[id] for id in train_index]
    
#     util.display_df(imp.pd.DataFrame([outer_ids, outer_labels]).T)
    df=imp.pd.DataFrame(dict(id=outer_ids))
    df.to_csv('{}cv_ids/outer_{}.csv'.format(c.directory['save'], outer_index), index=False)

    skf_inner = imp.StratifiedKFold(n_splits=c.inner, random_state=c.seed)
    for inner_index, (train_index, test_index) in enumerate(skf_inner.split(inner_ids, inner_labels)):
        inner_ids_final = [inner_ids[id] for id in test_index]
        inner_labels_final = [inner_labels[id] for id in test_index]

#         util.display_df(imp.pd.DataFrame([inner_ids_final, inner_labels_final]).T)
        df=imp.pd.DataFrame(dict(id=inner_ids_final))
        df.to_csv('{}cv_ids/inner_{}_{}.csv'.format(c.directory['save'], outer_index, inner_index), index=False)

Number of samples per each class = 5168
folder /Users/habibkarbasian/Documents/_Courses/_Datasets/_Retweetability/_ILLAE/cv_ids/ has been created.
